# Using rabbitmq-server Message Broker with Amazon AWS

### Installation

check notes on http://kronosapiens.github.io/blog/2015/04/28/rabbitmq-aws.html

Install rabbitmq-server server first:

In [1]:
#!sudo apt-get install rabbitmq-server

check if the rabbitmq server is up and runnning: 

In [2]:
!sudo rabbitmqctl status

Status of node 'rabbit@ip-172-31-9-236' ...
[{pid,1459},
 {running_applications,[{rabbit,"RabbitMQ","3.5.7"},
                        {mnesia,"MNESIA  CXC 138 12","4.13.3"},
                        {xmerl,"XML parser","1.3.10"},
                        {os_mon,"CPO  CXC 138 46","2.4"},
                        {sasl,"SASL  CXC 138 11","2.7"},
                        {stdlib,"ERTS  CXC 138 10","2.8"},
                        {kernel,"ERTS  CXC 138 10","4.2"}]},
 {os,{unix,linux}},
 {erlang_version,"Erlang/OTP 18 [erts-7.3] [source] [64-bit] [smp:4:4] [async-threads:64] [kernel-poll:true]\n"},
 {memory,[{total,42842672},
          {connection_readers,0},
          {connection_writers,0},
          {connection_channels,0},
          {connection_other,2712},
          {queue_procs,2712},
          {queue_slave_procs,0},
          {plugins,0},
          {other_proc,13334048},
          {mnesia,61920},
          {mgmt_db,0},
          {msg_index,46736},
          {other_ets,769824},
         

RabbitMQ comes with a “guest” user out of the box, but this user is not configured to accept external requests. This would be fine if you were planning on sending and consuming messages locally, but we are not.

The easiest way to configure RabbitMQ to accept external requests is to create a new user. Go ahead and run the following. (Feel free to chahnge the myguest username and password to anytihng you like):

In [3]:
!sudo rabbitmqctl add_user myguest myguestpwd
!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

Creating user "myguest" ...
Setting permissions for user "myguest" in vhost "/" ...


This will create a new user on the RabbitMQ server. We will use this username and password to configure the app-layer instance to send messages to Rabbit.

Let’s check to make sure the user was created correctly:

In [4]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


Pika is a pure-Python implementation of the AMQP 0-9-1 protocol that is used by RabbitMQ.

In [10]:
#!pip install pika

In [1]:
import pika

### Example for sending message on Localhost 

In [2]:
# create a connection to 'localhost'
connection_local = pika.BlockingConnection(pika.ConnectionParameters('localhost'))


In [3]:
# create a channel on this connection:
channel_local = connection_local.channel()

Before you run the next cell, run the 'recive_localhost' code as well. That program will wait for message

In [4]:
# send a message to this channel by specifiying a routing_key
channel_local.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World 1!')
print(" [x] Sent 'Hello World!'")

 [x] Sent 'Hello World!'


### Here is another way with setting up credentials

In [5]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
parameters =  pika.ConnectionParameters('localhost', port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

In [6]:
channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World 2!')
print(" [x] Sent 'Hello World!'")

 [x] Sent 'Hello World!'


### Example for sending message through web

Below I'm using a balance loader (PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com) that I set up on AWS. 
You should change it to your machine's address

Before you run the next cell, make sure AWS "Target groups" in the "Load balancing" are set up correctly and the instances are registered. 

In [2]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
parameters =  pika.ConnectionParameters('PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com', port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

Before you run the next cell, you should also run the receiver: "receive_through_url.ipynb"

In [5]:
channel.basic_publish(exchange='',
                      routing_key='hello2',
                      body='Hello World5!')
print(" [x] Sent 'Hello World!'")

 [x] Sent 'Hello World!'
